In [ ]:
import timeit
import pandas as pd
import numpy as np
import pandas as pd
import seaborn as sns
import os
import scipy.stats as stats
import tensorflow as tf
import matplotlib.pyplot as plt
import string
import nltk
import sklearn

from scipy.stats import entropy
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
from gensim.models import KeyedVectors
from sklearn.ensemble import RandomForestRegressor
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from transformers import AutoTokenizer, AutoModel, TFAutoModel
from sklearn.model_selection import cross_val_score, KFold

nltk.download('stopwords') # download stopwords corpus
nltk.download('punkt') # download punkt tokenizer



In [ ]:
# Transformer for reading CSV file into DataFrame
class CSVReader(BaseEstimator, TransformerMixin):
    def __init__(self, file_path):
        self.file_path = file_path
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        # Read the CSV file into a DataFrame
        df = pd.read_csv(self.file_path)
        
        # Return the DataFrame
        return df
    
# Transformer for Gensim Dutch Word2Vec embeddings
class GensimEmbeddings(BaseEstimator, TransformerMixin):
    def __init__(self, model_path, embedding_size):
        self.model_path = model_path
        self.embedding_size = embedding_size
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        # Load the Gensim Word2Vec model
        model = KeyedVectors.load_word2vec_format(self.model_path)
        
        # Initialize an empty numpy array to store the embeddings
        embeddings = np.zeros((len(X), self.embedding_size))
        
        # Iterate over the words in the text and calculate the embeddings
        for i, word in enumerate(X['prep_content']):
            if word in model:
                embeddings[i, :] = model[word]
            else:
                embeddings[i, :] = np.random.randn(self.embedding_size)
        
        # Add the embeddings to the DataFrame
        for j in range(embeddings.shape[1]):
            X[f'embedding_{j}'] = embeddings[:, j]
        
        return X

# Transformer for TFIDF embeddings
class TFIDFEmbeddings(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.tfidf_vectorizer = TfidfVectorizer()
    
    def fit(self, X, y=None):
        self.tfidf_vectorizer.fit(X['prep_content'])
        return self
    
    def transform(self, X):
        # Fit the TF-IDF vectorizer if it's not fitted
        if not hasattr(self.tfidf_vectorizer, 'idf_'):
            self.fit(X)
        
        # Calculate the TF-IDF embeddings
        tfidf_embeddings = self.tfidf_vectorizer.transform(X['prep_content'])

        # Add the embeddings to the DataFrame
        for i in range(tfidf_embeddings.shape[1]):
            X[f'tfidf_embedding_{i}'] = tfidf_embeddings[:, i].toarray().flatten()
        
        n_features = len(self.tfidf_vectorizer.vocabulary_)

        return X, n_features
    
class BERTEmbeddings(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.tokenizer = AutoTokenizer.from_pretrained("GroNLP/bert-base-dutch-cased")
        self.model = TFAutoModel.from_pretrained("GroNLP/bert-base-dutch-cased")  # Tensorflow
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        embeddings = np.zeros((len(X), 768))
        for i, text in enumerate(X['content']):
            input_ids = self.tokenizer.encode(str(text), add_special_tokens=True, return_tensors="tf")
            output = self.model(input_ids)
            embeddings[i] = np.max(output.last_hidden_state.numpy(), axis=1)
        
        for j in range(embeddings.shape[1]):
            X[f'embedding_{j}'] = embeddings[:, j]
        
        return X  
       
# Transformer for baseline model   
class BaselineAnalysis(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        avg_entropy_per_source = X.groupby('name')['entropy'].mean()
        X['avg_entropy'] = X['name'].map(avg_entropy_per_source)

        mse = mean_squared_error(X['entropy'], X['avg_entropy'])
        print("Baseline: Mean Squared Error:", mse)

        errors = X['entropy'] - X['avg_entropy']
        squared_errors = errors ** 2
        variance = np.var(squared_errors)
        print("Baseline: Variance of Squared Errors:", variance)

        std = np.std(squared_errors)
        print("Baseline: STD of Squared Errors:", std)
        
        # Bootstrapping to calculate confidence intervals around MSE
        n_iterations = 1000  # Number of bootstrap iterations
        mse_bootstrapped = []
        for _ in range(n_iterations):
            # Resample the squared errors
            resampled_errors = np.random.choice(squared_errors, size=len(squared_errors), replace=True)
            # Calculate MSE from resampled errors
            mse_bootstrapped.append(np.mean(resampled_errors))

        # Calculate the lower and upper percentiles for confidence interval
        lower_percentile = 10  # 2.5th percentile
        upper_percentile = 90  # 97.5th percentile
        ci_lower = np.percentile(mse_bootstrapped, lower_percentile)
        ci_upper = np.percentile(mse_bootstrapped, upper_percentile)
        print(f"Baseline: MSE Confidence Interval ({lower_percentile}%, {upper_percentile}%): ({ci_lower}, {ci_upper})")

        plt.figure(figsize=(8, 6))
        plt.hist(errors, bins=20, edgecolor='black', alpha=0.75)
        plt.xlabel('Error')
        plt.ylabel('Frequency')
        plt.title('Baseline: Distribution of Errors in Test Set')
        plt.show()

        return X 
    
# Transformer for Random Forest
class RFTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, embedding_option):
        self.embedding_option = embedding_option
        self.embedding_step = None
        self.xgboost_model = None
        self.y = None
    
    def fit(self, X, y= None):
        if self.embedding_option == 'gensim':
            self.embedding_step = GensimEmbeddings(model_path='INPUT EMBEDDING PATH', embedding_size=160)
            X = self.embedding_step.transform(X)
            # Prepare the input features
            X_features = X.iloc[:, -160:]
        elif self.embedding_option == 'tfidf':
            self.embedding_step = TFIDFEmbeddings()
            X, n_features = self.embedding_step.transform(X)
            X_features = X.iloc[:, -n_features:]
        elif self.embedding_option == 'bert':
            self.embedding_step = BERTEmbeddings()
            X = self.embedding_step.transform(X)
            # Prepare the input features
            X_features = X.iloc[:, -768:]
        
        # In the first step we will split the data in training and remaining dataset
        y_target = X['entropy']
        self.y = y_target
        self.X_train, self.X_rem, self.y_train, self.y_rem = train_test_split(X_features , y_target, train_size=0.8)

        # Now since we want the valid and test size to be equal (10% each of overall data). 
        # we have to define valid_size=0.5 (that is 50% of remaining data)

        self.X_valid, self.X_test, self.y_valid, self.y_test = train_test_split(self.X_rem, self.y_rem, test_size=0.5)
        
        self.rf_model = RandomForestRegressor(random_state=42)
        
        # define the parameter grid to search
        param_grid = {
            'n_estimators': [50, 100, 150],
            'max_depth': [5, 10, 15],
            'min_samples_split': [2, 5],
            'min_samples_leaf': [1, 3, 4, 5]
        }
        
        print("Performing grid search...")
        
        # create a GridSearchCV object and fit it to the data
        grid_search = GridSearchCV(RandomForestRegressor(random_state=42), param_grid=param_grid, cv=10, n_jobs=-1, scoring='neg_mean_squared_error')
        grid_search = grid_search.fit(self.X_train, self.y_train)
        pd.DataFrame(grid_search.cv_results_)

        # get the best model and print its hyperparameters
        self.rf_model = grid_search.best_estimator_

        return self
    
    def transform(self, X):
        if self.embedding_step is not None:
            X = self.embedding_step.transform(X)
        
        # evaluate the model on the test set
        y_pred = self.rf_model.predict(self.X_test)
        test_mse = mean_squared_error(self.y_test, y_pred)
        print("Random Forest: Test MSE (tuned):", test_mse)
        
        # Calculate the errors
        errors = self.y_test - y_pred
        
        # Calculate the variance and standard deviation of errors
        variance = np.var(errors)
        std = np.std(errors)
        print('Random Forest: Variance of Errors:', variance)
        print('Random Forest: Standard Deviation of Errors:', std)
        
        # Calculate the squared errors
        squared_errors = errors ** 2
        # Bootstrapping to calculate confidence intervals around MSE
        n_iterations = 1000  # Number of bootstrap iterations
        mse_bootstrapped = []
        for _ in range(n_iterations):
            # Resample the squared errors
            resampled_errors = np.random.choice(squared_errors, size=len(squared_errors), replace=True)
            # Calculate MSE from resampled errors
            mse_bootstrapped.append(np.mean(resampled_errors))

        # Calculate the lower and upper percentiles for confidence interval
        lower_percentile = 10  # 2.5th percentile
        upper_percentile = 90  # 97.5th percentile
        ci_lower = np.percentile(mse_bootstrapped, lower_percentile)
        ci_upper = np.percentile(mse_bootstrapped, upper_percentile)
        print(f"Random Forest: MSE Confidence Interval ({lower_percentile}%, {upper_percentile}%): ({ci_lower}, {ci_upper})")

        
        # Plot the distribution of errors
        plt.figure(figsize=(8, 6))
        plt.hist(errors, bins=20, edgecolor='black', color='purple', alpha=0.75)
        plt.xlabel('Error')
        plt.ylabel('Frequency')
        plt.title('Random Forest: Distribution of Errors in Test Set')
        plt.show()
        
        # Plot the distribution of entropy scores in the Train set
        plt.figure(figsize=(8, 6))
        plt.hist(self.y_train, bins=20, edgecolor='black', color='purple', alpha=0.75)
        plt.xlabel('Entropy')
        plt.ylabel('Frequency')
        plt.title('Random Forest: Distribution of Entropy Scores in Train Set')
        plt.show()
        
        # Plot the distribution of entropy scores in the Test set
        plt.figure(figsize=(8, 6))
        plt.hist(self.y_test, bins=20, edgecolor='black', color='purple', alpha=0.75)
        plt.xlabel('Entropy')
        plt.ylabel('Frequency')
        plt.title('Random Forest: Distribution of Entropy Scores in Test Set')
        plt.show()
    
        return X

chosen_embedding = 'bert'  

pipeline = Pipeline([
    ('csv_reader', CSVReader('INPUT FILE PATH')),
    ('baseline_analysis', BaselineAnalysis()),
    ('RF', RFTransformer(embedding_option=chosen_embedding)),
    
])

embedding_options = {
    'gensim': GensimEmbeddings(model_path='INPUT EMBEDDING PATH', embedding_size=160),
    'tfidf': TFIDFEmbeddings(),
    'bert': BERTEmbeddings(),
}

pipeline.set_params(RF__embedding_option=chosen_embedding)
df_transformed = pipeline.fit_transform(None)